## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, use_current_place_point_only, use_current_sub_binders_only, finish_L_shape, set_l_shape_object

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8267417 , -0.02449634, -0.5620481 ,  0.6496977 ],
        [ 0.17165463,  0.9623947 ,  0.2105494 , -0.15858173],
        [ 0.53575444, -0.27054814,  0.7998568 ,  0.09059656],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


## add indy tool

In [8]:
add_indy_sweep_tool(gscene, "indy0", face_name="brush_face")

## add sweep face

In [9]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [10]:
gscene.update_markers_all()

## add wp

In [11]:
TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.17
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))

In [12]:
gscene.update_markers_all()

## add blocking structure on table

In [13]:
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

## add box

In [14]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [15]:
gscene.update_markers_all()

## Register binders

In [16]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [17]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)

## add objects

In [18]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [19]:
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [20]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [21]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)

### planners

In [22]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [23]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [24]:
mplan.motion_filters = checkers_all

In [25]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [26]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

(0, 0, 0)
generate table - Geometry
generate table - Handle
generate table - Binder


In [27]:
pscene.subject_name_list

generate table - Object


['sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [28]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [29]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan & show

In [30]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()

In [31]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
    finish_L_shape(gscene, gtem_dict)

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        with gtimer.block(gname):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)
    set_l_shape_object(pscene)
initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
gscene.update_markers_all()

# remove place points and sub-binders except for the current ones
use_current_place_point_only(pscene, initial_state)
use_current_sub_binders_only(pscene, initial_state)
tplan.prepare()
mplan.update_gscene()

In [32]:
print(gtimer)

detect: 	164.0 ms/1 = 164.301 ms (164.301/164.301)

generate table - Marker
generate table - MarkerGroup
button clicked
button action done


In [34]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
    finish_L_shape(gscene, gtem_dict)

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    set_l_shape_object(pscene)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

('track_face', 0, 0, 0)
Use 36/36 agents
try: 0 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0)
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
try transition motion
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
try: 0 - ('track_fa

try: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
result: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = success
result: 7 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0) = fail
branching: 1->8 (1.05/50.0 s, steps/err: 42(227.967977524 ms)/0.00157930013668)
result: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
branching: 1->9 (1.08/50.0 s, steps/err: 7(206.79807663 ms)/0.00115964055313)
try: 9 - ('track_face', 0, 0, 0)->('track_face', 0, 0, 0)
result: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
try: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
try: 9 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0)
joint motion tried: True
try joint motion
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
result: 9 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0) = fail
try: 6 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
try: 6 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
try: 6 - ('grip1', 0, 0, 0)-

try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
branching: 1->20 (1.61/50.0 s, steps/err: 17(198.843002319 ms)/0.001218251721)
result: 6 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
try: 20 - ('track_face', 0, 0, 0)->('track_face', 0, 0, 0)
result: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
try constrained motion
try constrained motion
try: 6 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 6 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
transition motion tried: True
try: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
try: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
result: 1 - ('grip1', 0, 0, 0)->('track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = fail
try: 21 - ('track_face', 1, 0, 0)->('track_face', 2, 0, 0)
result: 0 - ('track_f

try: 25 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0)
result: 29 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0) = fail
transition motion tried: True
result: 29 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0) = fail
result: 26 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0) = fail
joint motion tried: True
++ adding return motion to acquired answer ++
result: 20 - ('track_face', 0, 0, 0)->('track_face', 0, 0, 0) = success
try: 31 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0)
try: 28 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0)
try: 25 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0)
try transition motion
try joint motion
try: 26 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0)
result: 0 - ('track_face', 0, 0, 0)->('grip1', 0, 0, 0) = success
try joint motion
try: 28 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0)
try: 33 - ('grip1', 0, 0, 0)->('grip1', 0, 0, 0)
result: 23 - ('track_face', 1, 0, 0)->('track_face', 2, 0, 0) = success
result: 25 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0) = fail

joint motion tried: True
======================= terminated 14: first answer acquired from other agent ===============================
======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired from other agent ===============================
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 0: first answer acquired ===============================
constrained motion tried: True
branching: 30->42 (2.45/50.0 s, steps/err: 33(415.961027145 ms)/0.000982168975015)
result: 30 - ('track_face', 1, 0, 0)->('track_face', 2, 0, 0) = success
++ adding return motion to acquired answer ++
======================= terminated 31: first answer acquired from other agent ===============================
transition motion tried: True
try joint motion
result: 28 - ('track_face', 2, 0, 0)->('grip1', 2, 0, 0)

...

result: 4 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2) = fail
transition motion tried: True
transition motion tried: True
transition motion tried: False
joint motion tried: True
result: 2 - ('track_face', 2, 2, 0)->('track_face', 2, 2, 0) = success
result: 5 - ('track_face', 2, 2, 0)->('track_face', 2, 2, 1) = success
result: 0 - ('grip1', 2, 2, 0)->('track_face', 2, 2, 0) = success
try: 4 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2)
result: 3 - ('track_face', 2, 2, 0)->('track_face', 2, 2, 1) = success
try: 4 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2)
try: 4 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2)
try: 4 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2)
result: 31 - ('track_face', 0, 0, 0)->('track_face', 1, 0, 0) = fail
result: 0 - ('grip1', 2, 2, 0)->('track_face', 2, 2, 0) = success
branching: 0->8 (1.13/50.0 s, steps/err: 62(230.98897934 ms)/0.00170428349066)
result: 4 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2) = fail
joint motion tried: True
branching: 0->10 (1.14/50.0 

try: 16 - ('track_face', 2, 2, 1)->('track_face', 2, 2, 2)
++ adding return motion to acquired answer ++
branching: 12->17 (1.54/50.0 s, steps/err: 31(126.518011093 ms)/0.00195396577232)
try: 14 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2)
try: 17 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2)
try: 14 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2)
try: 1 - ('grip1', 2, 2, 1)->('grip1', 2, 2, 2)
try transition motion
try: 14 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2)
try: 14 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2)
++ adding return motion to acquired answer ++
result: 16 - ('track_face', 2, 2, 1)->('track_face', 2, 2, 2) = fail
try joint motion
result: 14 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2) = fail
try: 14 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2)
result: 17 - ('track_face', 2, 2, 2)->('grip1', 2, 2, 2) = fail
++ adding return motion to acquired answer ++
joint motion tried: True
try joint motion
try: 1 - ('grip1', 2, 2, 1)->('grip1', 2, 2, 2)
try: 17 - ('track_face'

branching: 1->29 (2.03/50.0 s, steps/err: 30(290.364027023 ms)/0.000852570961709)
======================= terminated 25: first answer acquired from other agent ===============================
======================= terminated 27: first answer acquired from other agent ===============================
======================= terminated 26: first answer acquired from other agent ===============================
result: 18 - ('track_face', 2, 2, 0)->('track_face', 2, 2, 1) = success
======================= terminated 15: first answer acquired ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 22: first answer acquired from other agent ===============================
joint motion tried: True
constrained motion tried: True
transition motion tried: True
branching: 18->31 (2.07/50.0 s, steps/err: 41(294.416189194 ms)/0.00166278665029)
======================= terminated 

======================= terminated 34: first answer acquired from other agent ===============================
======================= terminated 35: first answer acquired from other agent ===============================
======================= terminated 11: first answer acquired ===============================
======================= terminated 4: first answer acquired ===============================
transition motion tried: True
result: 7 - ('grip1', 2, 2, 0)->('track_face', 2, 2, 0) = success
branching: 7->21 (5.73/50.0 s, steps/err: 102(3359.39979553 ms)/0.00169963938809)
++ adding return motion to acquired answer ++
try joint motion
joint motion tried: True
try joint motion
transition motion tried: True
result: 20 - ('grip1', 2, 2, 2)->('track_face', 2, 2, 2) = success
branching: 20->38 (2.94/50.0 s, steps/err: 99(1090.12007713 ms)/0.00173815706838)
++ adding return motion to acquired answer ++
try joint motion
joint motion tried: True
joint motion tried: True
====================

Process Process-77:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 157, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 204, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 102, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 270, in plan_algorithm
    timeout=timeout, **kwargs)
  File "pkg/planning/motion/moveit/moveit_py.py", line 116, in plan_py
    JointState(self.joint_num, *Q_init), plannerconfig, timeout)
KeyboardInterrupt


In [35]:
print(gtimer)

detect: 	830.0 ms/1 = 829.981 ms (829.981/829.981)
firstmove: 	3094.0 ms/1 = 3094.459 ms (3094.459/3094.459)
plan0: 	2389.0 ms/1 = 2389.474 ms (2389.474/2389.474)
initialize_memory: 	216.0 ms/3 = 72.08 ms (12.593/188.535)
init_search: 	60.0 ms/3 = 20.027 ms (17.17/21.854)
start_process: 	6953.0 ms/3 = 2317.593 ms (2002.082/2607.747)
plan1: 	2648.0 ms/1 = 2647.74 ms (2647.74/2647.74)
plan2: 	2228.0 ms/1 = 2228.382 ms (2228.382/2228.382)



## mix full schedule

In [36]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

mix_schedule_safe: 	69.0 ms/1 = 69.193 ms (69.193/69.193)



### play schedule

In [37]:
ppline.play_schedule(safe_mixed, period=0.001)

('track_face', 0, 0, 0)->('track_face', 0, 0, 0)
('track_face', 0, 0, 0)->('grip1', 0, 0, 0)
('grip1', 0, 0, 0)->('track_face', 0, 0, 0)
('track_face', 0, 0, 0)->('track_face', 1, 0, 0)
('track_face', 1, 0, 0)->('track_face', 2, 0, 0)
('track_face', 2, 0, 0)->('grip1', 2, 0, 0)
('grip1', 2, 0, 0)->('grip1', 2, 1, 0)
('grip1', 2, 1, 0)->('grip1', 2, 2, 0)
('grip1', 2, 2, 0)->('track_face', 2, 2, 0)
('track_face', 2, 2, 0)->('track_face', 2, 2, 1)
('track_face', 2, 2, 1)->('track_face', 2, 2, 2)
('track_face', 2, 2, 2)->('track_face', 2, 2, 2)


### Run fully mixed schedule

In [39]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(False,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(2)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

connection command:
indy0: False
panda1: True


In [44]:
ppline.execute_schedule(safe_mixed)

binder: grip1
rname: panda1
binder: None
binder: None
binder: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: track_face
rname: N

## Plan & execute

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
for _ in range(3):
    indy = crob.robot_dict['indy0']
    with indy:
        indy.wait_di(16)

    mplan.reset_log(False)
    gtimer.reset()

    with gtimer.block("detect"):
        for sname in pscene.subject_name_list:
            if isinstance(pscene.subject_dict[sname], AbstractObject):
                pscene.remove_subject(sname)
            
        gscene.clear_non_fixed()

        ## detect again
        gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
        finish_L_shape(gscene, gtem_dict)
    
        ## create boxes
        box_list =  [] 
        for gname in sorted(gtem_dict.keys()):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)
        set_l_shape_object(pscene)

        initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
        gscene.update_markers_all()

        # remove place points and sub-binders except for the current ones
        use_current_place_point_only(pscene, initial_state)
        use_current_sub_binders_only(pscene, initial_state)
        tplan.prepare()
        mplan.update_gscene()

    print(initial_state.node)

    gtimer.tic("firstmove")
    obj_num = len(gtem_dict)
    sweep_num = len(sweep_list)
    from_state = initial_state
    t_exe = None
    snode_schedule_all = []
    for sweep_idx in range(sweep_num):
        gcheck.put_banned = [track_list[sweep_idx][2]]
        sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
    #     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
        goal_nodes = [("track_face",)*obj_num+sweep_goal]
        if sweep_idx < sweep_num-1:
            for i_s in range(obj_num):
                obj_goal = ["track_face"]*obj_num
                obj_goal[i_s] = "grip1"
                goal_nodes += [tuple(obj_goal)+sweep_goal]
        gtimer.tic("plan{}".format(sweep_idx))
        ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                      timeout_loop=20, multiprocess=True, timeout=5,
                      plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
        gtimer.toc("plan{}".format(sweep_idx))
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.tplan.sort_schedule(schedules)
        snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
        snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
        snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
        snode_schedule = mix_schedule(mplan, snode_schedule_safe)
        from_state = snode_schedule[-1].state
        if t_exe:
            t_exe.join()
        else:
            gtimer.toc("firstmove")
        t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), kwargs=dict(auto_stop=False))
        t_exe.start()
        snode_schedule_all.append(snode_schedule)
    t_exe.join()
    crob.stop_tracking()

In [37]:
print(gtimer)

detect: 	175.0 ms/1 = 174.81 ms (174.81/174.81)
firstmove: 	4552.0 ms/1 = 4551.953 ms (4551.953/4551.953)
plan0: 	3496.0 ms/1 = 3495.868 ms (3495.868/3495.868)
initialize_memory: 	22.0 ms/3 = 7.271 ms (6.438/8.883)
init_search: 	43.0 ms/3 = 14.271 ms (10.722/19.792)
start_process: 	1842.0 ms/3 = 613.957 ms (550.054/669.434)
plan1: 	3133.0 ms/1 = 3132.877 ms (3132.877/3132.877)
plan2: 	5273.0 ms/1 = 5273.061 ms (5273.061/5273.061)

